### VII. Performing Arithmetic Operations
Explore advanced arithmetic operations, including bit shifting, multiplication, division, and dealing with overflow and underflow. You'll also learn to extend basic operations using carry and borrow and manipulate decimal values.

## Topics Covered
A. How and why to use shift and rotate

B. Multiplication and over ow

C. Division and remainder

D. Expanding simple addition with the carry instruction

E. Expanding simple subtraction with the borrow instruction

F. Arithmetic operations on decimal variables

### **A. How and Why to Use Shift and Rotate**

Assembly language offers **bit-level operations** that can manipulate data quickly and efficiently. Two commonly used operations are **Shift** and **Rotate**.

---

### 🔹 What is **Shift**?

A **shift** operation moves bits in a register **left or right**, inserting 0s (or the sign bit) on one side and **discarding** the bits on the other side.

#### 🔸 Types of Shift Instructions

| Instruction | Description            | Example Effect          |
| ----------- | ---------------------- | ----------------------- |
| `SHL` (SAL) | Shift Left Logical     | Multiplies by 2         |
| `SHR`       | Shift Right Logical    | Divides by 2 (unsigned) |
| `SAR`       | Shift Arithmetic Right | Divides by 2 (signed)   |

#### 💡 Why Use Shift?

* To **multiply or divide** integers by powers of 2 (very fast).
* To **manipulate individual bits** (e.g., extract flags).
* For **encryption, compression**, and other low-level bit tricks.

---

### 🔹 What is **Rotate**?

A **rotate** operation is similar to shift, but instead of discarding the bit that moves out, it wraps around and comes back on the other end.

#### 🔸 Types of Rotate Instructions

| Instruction | Description                |
| ----------- | -------------------------- |
| `ROL`       | Rotate Left                |
| `ROR`       | Rotate Right               |
| `RCL`       | Rotate Left through Carry  |
| `RCR`       | Rotate Right through Carry |

---

### 🛠 Example Code (NASM-style)

In [ ]:
section .data
    num db 0b11001010     ; 202 in binary

section .text
    global _start

_start:
    mov al, [num]         ; Load byte into AL
    shl al, 1             ; Shift left by 1 (multiply by 2)
    ; AL now = 0b10010100 = 0x94

    mov al, [num]
    shr al, 1             ; Shift right by 1 (divide by 2)
    ; AL now = 0b01100101 = 0x65

    mov al, [num]
    rol al, 1             ; Rotate left
    ; AL now = 0b10010101 = 0x95

    mov al, [num]
    ror al, 1             ; Rotate right
    ; AL now = 0b01100101 = 0x65

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall

### 📌 Notes:

* Use `SHL` and `SHR` for **arithmetic-like** operations.
* Use `ROL` and `ROR` when you want to **preserve all bits** and just reorder them.
* **Carry flag (CF)** can also be involved when using `RCL`, `RCR`.


⬇️

### **B. Multiplication and Overflow**

Multiplication in assembly is a powerful operation but must be handled carefully because results can easily overflow the destination register.

---

### 🔹 Unsigned Multiplication - `MUL`

The `MUL` instruction multiplies unsigned integers.

* Operand is a register or memory.
* Multiplies `AL`, `AX`, or `EAX` (depending on operand size) by the operand.
* The result is stored in a **wider register** (e.g., AX x BX → DX\:AX).

#### 🧠 Register Pairs:

| Operand Size | Multiplies  | Result   |
| ------------ | ----------- | -------- |
| 8-bit        | AL x r/m8   | AX       |
| 16-bit       | AX x r/m16  | DX\:AX   |
| 32-bit       | EAX x r/m32 | EDX\:EAX |

#### 🔸 Example (Unsigned)

```nasm
mov al, 10        ; AL = 10
mov bl, 20        ; BL = 20
mul bl            ; AX = AL * BL → AX = 200
```

---

### 🔹 Signed Multiplication - `IMUL`

The `IMUL` instruction is used for **signed integers**.

* Handles negative numbers properly.
* Can use one, two, or three operands.

#### Forms of `IMUL`:

1. `IMUL reg` - single operand (similar to `MUL`)
2. `IMUL reg, reg/mem` - two operands
3. `IMUL reg, reg/mem, imm` - three operands

#### 🔸 Example (Signed)

```nasm
mov ax, -5
mov bx, 4
imul bx           ; AX = AX * BX = -20
```

---

### ⚠️ Detecting **Overflow**

Overflow means the result is **too large** to fit in the destination register.

* **Carry Flag (CF)** and **Overflow Flag (OF)** are set if overflow occurs.
* Always check these flags after multiplication, especially for unsigned `MUL`.

#### 🔍 Checking for Overflow (Simple Example)

```nasm
mov al, 255
mov bl, 2
mul bl            ; 255 * 2 = 510 → won't fit in AL, stored in AX

jc overflow       ; Jump if Carry is set (overflow)
; otherwise continue

overflow:
; handle overflow here
```

---

### 🧮 Why Use `IMUL` vs `MUL`?

| Instruction | Use Case        | Handles Negatives? |
| ----------- | --------------- | ------------------ |
| `MUL`       | Unsigned values | ❌ No               |
| `IMUL`      | Signed values   | ✅ Yes              |

---

### 🛠 Summary

* Use `MUL` for positive values, `IMUL` when negative values are involved.
* Always be aware of **result size** and potential **overflow**.
* Handle large results using `EDX:EAX`, `DX:AX`, etc.


⬇️

### **C. Division and Remainder**

Division in assembly is handled using the `DIV` (for unsigned numbers) and `IDIV` (for signed numbers) instructions. Just like multiplication, the result can span multiple registers, and errors like **divide-by-zero** must be avoided.

---

### 🔹 Unsigned Division - `DIV`

The `DIV` instruction divides a **larger dividend** by a smaller divisor.

| Operand Size | Dividend | Divisor | Quotient | Remainder |
| ------------ | -------- | ------- | -------- | --------- |
| 8-bit        | AX       | r/m8    | AL       | AH        |
| 16-bit       | DX\:AX   | r/m16   | AX       | DX        |
| 32-bit       | EDX\:EAX | r/m32   | EAX      | EDX       |

#### 🔸 Example: 8-bit Division

```nasm
mov ax, 50        ; AX holds the dividend (50)
mov bl, 3         ; BL is the divisor
div bl            ; AL = 16 (quotient), AH = 2 (remainder)
```

---

### 🔹 Signed Division - `IDIV`

* `IDIV` works the same as `DIV`, but for **signed integers**.
* The dividend must be **sign-extended** beforehand.

#### 🔸 Sign Extension Instructions

| Instruction | Purpose                          |
| ----------- | -------------------------------- |
| `CBW`       | AL → AX (8-bit → 16-bit)         |
| `CWD`       | AX → DX\:AX (16-bit → 32-bit)    |
| `CDQ`       | EAX → EDX\:EAX (32-bit → 64-bit) |

#### 🔸 Example: Signed Division

```nasm
mov ax, -30       ; AX = -30
cwd               ; Sign-extend AX into DX:AX
mov bx, 7
idiv bx           ; AX = -4 (quotient), DX = -2 (remainder)
```

---

### ⚠️ Common Pitfalls

* 💥 **Divide-by-zero error**: Happens if divisor is 0 — causes a CPU exception.
* ❌ **Wrong size register**: The dividend must always be in the correct register pair (e.g., AX or DX\:AX).
* ⚠️ **Signed division** must **sign-extend** the dividend to avoid incorrect results.

---

### 🛠 Practical Example: Division and Remainder

```nasm
mov ax, 19        ; Dividend
mov bl, 5         ; Divisor
div bl            ; AL = 3 (quotient), AH = 4 (remainder)

; Result:
; Quotient in AL = 3
; Remainder in AH = 4
```

---

### 📌 Summary

| Instruction | Use For  | Handles Negatives? | Result Registers                            |
| ----------- | -------- | ------------------ | ------------------------------------------- |
| `DIV`       | Unsigned | ❌ No               | Quotient: AL/AX/EAX<br>Remainder: AH/DX/EDX |
| `IDIV`      | Signed   | ✅ Yes              | Same as above                               |

* Always prepare the dividend in the correct register size.
* Always check that the divisor is **not zero**.
* Use `CBW`, `CWD`, or `CDQ` for proper **sign-extension** before `IDIV`.


⬇️

### **D. Expanding Simple Addition with the Carry Instruction**

When performing **multi-byte or multi-word addition** in Assembly, the standard `ADD` instruction is **not enough** on its own. That's where the **`ADC` (Add with Carry)** instruction comes in.

It allows us to chain additions together - like adding two 32-bit or 64-bit numbers by **processing them one part at a time**, starting from the **least significant byte** and carrying any overflow into the next.

---

### 🔹 Instructions Involved

| Instruction | Description                    |
| ----------- | ------------------------------ |
| `ADD`       | Adds two operands              |
| `ADC`       | Adds two operands + Carry flag |

---

### 🧠 Concept

Imagine adding two 16-bit numbers manually:

```
  A1A2
+ B1B2
-------
```

You add **A2 + B2**, then if there's a carry, you add that to **A1 + B1**.

Assembly does the same:

* Use `ADD` for least significant byte.
* Use `ADC` for most significant byte (to include carry from previous `ADD`).

---

### 🛠 Example: 16-bit Addition Using `ADC`

We'll add:

* `0x00FF` (255)
* `0x0002` (2)

Expected result: `0x0101` (257)

---

### ✅ Example

In [ ]:
# Save as: add_with_carry.asm
!echo '
section .data
    num1 dw 0x00FF        ; First 16-bit number (low: FF, high: 00)
    num2 dw 0x0002        ; Second 16-bit number

section .bss
    result_low resb 1
    result_high resb 1

section .text
    global _start

_start:
    ; Load lower bytes
    mov al, byte [num1]
    add al, byte [num2]   ; AL = FF + 02 = 0x101 → AL = 0x01, CF = 1
    mov [result_low], al

    ; Load higher bytes
    mov al, byte [num1+1]
    adc al, byte [num2+1] ; AL = 00 + 00 + carry = 1
    mov [result_high], al

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall
' > add_with_carry.asm

# Assemble and link
!nasm -f elf64 add_with_carry.asm -o add_with_carry.o
!ld add_with_carry.o -o add_with_carry
!./add_with_carry

# Print result
!xxd -g1 -c2 add_with_carry.asm | grep result

### 🔍 Code Explanation

* **`mov al, [num1]`**: Load the lower byte of `num1`.
* **`add al, [num2]`**: Add the lower byte of `num2`. If result > 255, Carry Flag is set.
* **`adc al, [num2+1]`**: Add the higher byte + any carry from the previous step.
* **`result_low` and `result_high`** store the final result split into bytes.

So:
`0x00FF` + `0x0002` →

* Low byte: `FF + 02 = 101 (carry 1) → 0x01`
* High byte: `00 + 00 + carry = 1 → 0x01`
  Result: **0x0101**

---

### 📌 Summary

* Use `ADD` for the **lowest byte** addition.
* Use `ADC` for all higher bytes to include carry.
* Useful in **multi-byte arithmetic**, such as 32-bit or 64-bit math on an 8-bit CPU.


⬇️

### **E. Expanding Simple Subtraction with the Borrow Instruction**

Just like how `ADC` (Add with Carry) is used to expand addition to multiple bytes, Assembly uses **`SBB` (Subtract with Borrow)** for **multi-byte subtraction**.

---

### 🔹 Instructions Involved

| Instruction | Description                        |
| ----------- | ---------------------------------- |
| `SUB`       | Subtracts two values               |
| `SBB`       | Subtracts two values + Borrow flag |

---

### 🧠 Why Use `SBB`?

When subtracting multi-byte values, you:

1. Start with the **least significant byte** using `SUB`.
2. Use `SBB` on the higher bytes to subtract the **borrow (carry)** if it occurred.

This ensures that the subtraction is **mathematically accurate** across all byte parts of the number.

---

### 🛠 Example: 16-bit Subtraction Using `SBB`

We'll subtract:

* `0x0101` (257)
* `0x0002` (2)
  Expected result: `0x00FF` (255)

---

### ✅ Example

In [ ]:
# Save as: sub_with_borrow.asm
!echo '
section .data
    num1 dw 0x0101        ; First number: 257
    num2 dw 0x0002        ; Second number: 2

section .bss
    result_low resb 1
    result_high resb 1

section .text
    global _start

_start:
    ; Subtract lower byte
    mov al, byte [num1]
    sub al, byte [num2]       ; AL = 0x01 - 0x02 = 0xFF (with borrow)
    mov [result_low], al

    ; Subtract higher byte with borrow
    mov al, byte [num1+1]
    sbb al, byte [num2+1]     ; AL = 0x01 - 0x00 - borrow = 0x00
    mov [result_high], al

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall
' > sub_with_borrow.asm

# Assemble and link
!nasm -f elf64 sub_with_borrow.asm -o sub_with_borrow.o
!ld sub_with_borrow.o -o sub_with_borrow
!./sub_with_borrow

# Print result bytes
!xxd -g1 -c2 sub_with_borrow.asm | grep result

### 🔍 Code Explanation

* `sub al, [num2]`: Subtracts the low bytes. Result: `1 - 2 = -1 = 0xFF` (borrow occurs).
* `sbb al, [num2+1]`: Subtracts high bytes **including** the borrow. `1 - 0 - 1 = 0x00`.
* Final result is: `0x00FF` stored in `result_high` and `result_low`.

---

### ⚠️ Notes

* The **Carry Flag (CF)** is used as a **borrow indicator** in subtraction.
* **SBB** subtracts:
  `destination = destination - source - CF`
* Always use `SBB` on the higher bytes when chaining subtractions.

---

### 📌 Summary

| Instruction | Use For    | Description                      |
| ----------- | ---------- | -------------------------------- |
| `SUB`       | Low byte   | Basic subtraction                |
| `SBB`       | High bytes | Subtraction + borrow if occurred |


⬇️

### **F. Arithmetic Operations on Decimal Variables**

Assembly language supports **BCD (Binary-Coded Decimal)** arithmetic - useful when working with **decimal values** like in financial apps or calculators, where precision matters.

Normally, arithmetic is done in **binary**, but if you want results in proper **decimal digits (0-9)**, you'll need to adjust after addition or subtraction using special **adjust instructions**.

---

### 🔹 What is BCD?

BCD stands for **Binary-Coded Decimal**:

* Each **4 bits** (nibble) represent one decimal digit.
* For example: `0x25` = **decimal 25** (not 37!).

So:

* `0x12` (BCD) = 12 in decimal (not 18)
* `0x99` (BCD) = 99 in decimal (valid)
* `0x1A` is invalid (A = 10)

---

### 🔹 Key BCD Adjustment Instructions

| Instruction | Meaning                          | Used After    |
| ----------- | -------------------------------- | ------------- |
| `DAA`       | Decimal Adjust after Addition    | `ADD`         |
| `DAS`       | Decimal Adjust after Subtraction | `SUB`         |
| `AAA`       | ASCII Adjust after Addition      | `ADD` (ASCII) |
| `AAS`       | ASCII Adjust after Subtraction   | `SUB` (ASCII) |

---

### ✅ Let's Focus on `DAA` and `DAS`

#### `DAA` (Decimal Adjust AL after Addition)

Used **after adding BCD numbers**. It checks if the result in `AL` is a **valid BCD**, and if not, it adjusts it.

---

### 🛠 Example: Adding Two BCD Values (Colab-ready)

We'll add:
`0x25` (decimal 25) + `0x37` (decimal 37)
Expected result: **0x62** (decimal 62)

---

#### ✅ Full Code with `DAA`:

In [ ]:
# Save as: bcd_add.asm
!echo '
section .data
    bcd1 db 0x25
    bcd2 db 0x37

section .bss
    result db 0

section .text
    global _start

_start:
    mov al, [bcd1]
    add al, [bcd2]      ; AL = 0x25 + 0x37 = 0x5C
    daa                 ; Adjust AL to valid BCD → AL = 0x62
    mov [result], al

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall
' > bcd_add.asm

# Assemble and link
!nasm -f elf64 bcd_add.asm -o bcd_add.o
!ld bcd_add.o -o bcd_add
!./bcd_add

# View result byte
!xxd -g1 -c1 bcd_add.asm | grep result

### 🔍 Code Explanation

* `add al, [bcd2]`: Adds two BCD digits.
* `daa`: Corrects result to a valid decimal format if needed.
* `AL` = 0x62 = **Decimal 62** (correct).

---

### 🛠 Subtraction with `DAS`

Let's subtract:
`0x62` - `0x25` = **Decimal 37** = `0x37`

---

#### ✅ Full Code with `DAS`:

In [ ]:
# Save as: bcd_sub.asm
!echo '
section .data
    bcd1 db 0x62
    bcd2 db 0x25

section .bss
    result db 0

section .text
    global _start

_start:
    mov al, [bcd1]
    sub al, [bcd2]      ; AL = 0x62 - 0x25 = 0x3D
    das                 ; Adjust to valid BCD → AL = 0x37
    mov [result], al

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall
' > bcd_sub.asm

# Assemble and run
!nasm -f elf64 bcd_sub.asm -o bcd_sub.o
!ld bcd_sub.o -o bcd_sub
!./bcd_sub

# View result byte
!xxd -g1 -c1 bcd_sub.asm | grep result

### 📌 Summary

| Instruction | When to Use         | What It Does                        |
| ----------- | ------------------- | ----------------------------------- |
| `DAA`       | After BCD `ADD`     | Adjusts AL to valid decimal format  |
| `DAS`       | After BCD `SUB`     | Adjusts AL to valid decimal format  |
| `AAA/AAS`   | For ASCII BCD input | Use only if you're doing ASCII math |

* These instructions make it easier to work with **decimal-based systems**.
* Rarely used in modern systems, but important in **legacy hardware**, calculators, POS systems, etc.